In [29]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sys

aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

def num_costly_codon_pairs(seq, do_print):
    num = 0
    to_print = ""
    for i in range(0,(len(seq)-3),3):
        #if (seq[i+2] == "C" and seq[i+3] == "G"):
        if (seq[i+2] == "T" and seq[i+3] == "A" and ((seq[i+4] == "A" or seq[i+4] == "G"))):
            num += 1
            if (do_print == 1):
                to_print += str(i) + " " #seq[i:i+6] + " " + 
    if (do_print):
        print(to_print)
    return(num)

original_codons = "ATGTCGAAAGCTACATATAAGGAACGTGCTGCTACTCATCCTAGTCCTGTTGCTGCCAAG"\
                "CTATTTAATATCATGCACGAAAAGCAAACAAACTTGTGTGCTTCATTGGATGTTCGTACC"\
                "ACCAAGGAATTACTGGAGTTAGTTGAAGCATTAGGTCCCAAAATTTGTTTACTAAAAACA"\
                "CATGTGGATATCTTGACTGATTTTTCCATGGAGGGCACAGTTAAGCCGCTAAAGGCATTA"\
                "TCCGCCAAGTACAATTTTTTACTCTTCGAAGACAGAAAATTTGCTGACATTGGTAATACA"\
                "GTCAAATTGCAGTACTCTGCGGGTGTATACAGAATAGCAGAATGGGCAGACATTACGAAT"\
                "GCACACGGTGTGGTGGGCCCAGGTATTGTTAGCGGTTTGAAGCAGGCGGCGGAAGAAGTA"\
                "ACAAAGGAACCTAGAGGCCTTTTGATGTTAGCAGAATTGTCATGCAAGGGCTCCCTAGCT"\
                "ACTGGAGAATATACTAAGGGTACTGTTGACATTGCGAAGAGCGACAAAGATTTTGTTATC"\
                "GGCTTTATTGCTCAAAGAGACATGGGTGGAAGAGATGAAGGTTACGATTGGTTGATTATG"\
                "ACACCCGGTGTGGGTTTAGATGACAAGGGAGACGCATTGGGTCAACAGTATAGAACCGTG"\
                "GATGATGTGGTCTCTACAGGATCTGACATTATTATTGTTGGAAGAGGACTATTTGCAAAG"\
                "GGAAGGGATGCTAAGGTAGAGGGTGAACGTTACAGAAAAGCAGGCTGGGAAGCATATTTG"\
                "AGAAGATGCGGCCAGCAAAACTAA"
original_AA = "MSKATYKERAATHPSPVAAKLFNIMHEKQTNLCASLDVRTTKELLELVEALGPKICLLKT"\
                    "HVDILTDFSMEGTVKPLKALSAKYNFLLFEDRKFADIGNTVKLQYSAGVYRIAEWADITN"\
                    "AHGVVGPGIVSGLKQAAEEVTKEPRGLLMLAELSCKGSLATGEYTKGTVDIAKSDKDFVI"\
                    "GFIAQRDMGGRDEGYDWLIMTPGVGLDDKGDALGQQYRTVDDVVSTGSDIIIVGRGLFAK"\
                    "GRDAKVEGERYRKAGWEAYLRRCGQQN*"
store_best = [None] * len(original_AA)
for i in range(0,len(original_AA)):
    store_best[i] = dict()
    for codon in aa_to_syn_codons_dict[original_AA[i]]:
        store_best[i][codon] = 0
                
# do the last AA
for codon in store_best[-1]:
    store_best[i][codon] = (0,"")


# do the dynamic program, stating from the one before the end of the list, until the end
for pos in reversed(range(0, len(store_best)-1)):
    for codon in store_best[pos]:
        # consider as cost the cost of adding this codon to one of the best sequences
        # from the previous suffix
        # maximizer the dimers
        best_prev = ""
        best_cost = -1 #sys.maxsize #-1
        for prev_codon in store_best[pos+1]:
            cost = num_costly_codon_pairs(codon+prev_codon,0) + store_best[pos+1][prev_codon][0]
            # maximizer the dimers
            if ((cost > best_cost) or # > <
                ((cost == best_cost) and (prev_codon == original_codons[(pos+1)*3:(pos+1)*3+3]))): #try to keep same seq
                best_cost = cost
                best_prev = prev_codon
        store_best[pos][codon] = (best_cost, best_prev)
        #print("storing for " + str(pos) + " " + codon + ": " + str(best_cost) + " " + best_prev)

recoded_sequence = ""
next_codon = ""
for pos in range(0, len(store_best)):
    if pos == 0:
        # maximum
        best_first = ""
        best_first_cost = -1 # sys.maxsize #-1
        for codon in store_best[0]:
            # maximum
            if ((store_best[pos][codon][0] > best_first_cost) or  #> <
                ((cost == best_cost) and (codon == original_codons[(pos+1)*3:(pos+1)*3+3]))): #try to keep same seq
                best_first = codon
                best_first_cost = store_best[pos][codon][0]
                next_codon = store_best[pos][codon][1]
        recoded_sequence += best_first
    else:
        recoded_sequence += next_codon
        next_codon = store_best[pos][next_codon][1]

#print(original_codons)
print(recoded_sequence)    
print("num costly codon pairs: " + str(num_costly_codon_pairs(recoded_sequence, 1)))
back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != original_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (original_AA == back_to_original):
    print("AA sequence remains the same, good")
print(original_codons)
print("num costly codon pairs in original sequences: " + str(num_costly_codon_pairs(original_codons, 1)))

ATGTCTAAAGCTACATATAAGGAACGTGCTGCTACTCATCCTAGTCCTGTTGCTGCTAAGCTATTTAATATCATGCACGAAAAGCAAACTAACTTGTGTGCTAGCTTGGATGTTAGAACCACTAAGGAATTACTGGAGTTAGTTGAAGCATTAGGTCCTAAAATTTGTTTACTTAAAACACATGTGGATATCTTGACTGATTTTAGCATGGAGGGCACAGTTAAGCCGCTTAAGGCACTTAGCGCTAAGTATAATTTTTTACTCTTCGAAGACCGTAAATTTGCTGACATTGGTAATACAGTTAAATTGCAGTATAGCGCGGGTGTATATAGAATAGCAGAATGGGCAGACATTACTAATGCACACGGTGTGGTGGGCCCAGGTATTGTTAGCGGTCTTAAGCAGGCGGCGGAAGAAGTAACTAAGGAACCTAGAGGCCTTTTGATGTTAGCAGAACTTAGCTGTAAGGGTAGCCTAGCTACTGGAGAATATACTAAGGGTACTGTTGACATTGCTAAGAGCGATAAAGATTTTGTTATCGGCTTTATTGCTCAAAGAGACATGGGTGGTAGAGATGAAGGTTACGATTGGTTGATTATGACACCCGGTGTGGGTTTAGATGATAAGGGAGACGCATTGGGTCAACAGTATAGAACCGTGGATGATGTGGTTAGCACAGGTAGCGACATTATTATTGTTGGTAGAGGACTATTTGCTAAGGGTAGGGATGCTAAGGTAGAGGGTGAACGTTACCGTAAAGCAGGCTGGGAAGCATATTTGCGTAGATGCGGCCAGCAAAACTAA
3 15 39 54 63 87 99 111 120 156 171 201 219 228 237 243 249 273 291 300 312 327 354 387 396 420 429 456 462 468 492 513 522 567 621 648 669 678 699 714 720 729 753 780 
num costly codon pairs: 44